In [1]:
%config Completer.use_jedi = False

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv3D, ConvLSTM2D, UpSampling2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os

In [2]:
## X:
x_train = np.load("/opt/datos/dataset/gfs/lstm/x_train_8lags.npy")
x_val   = np.load("/opt/datos/dataset/gfs/lstm/x_val_8lags.npy")
print(x_train.shape)
print(x_val.shape)

## Y:
y_train = np.load("/opt/datos/dataset/gfs/lstm/y_train_8lags.npy")
y_val   = np.load("/opt/datos/dataset/gfs/lstm/y_val_8lags.npy")
print(y_train.shape)
print(y_val.shape)

(13496, 4, 128, 144, 1)
(2892, 4, 128, 144, 1)
(13496, 4, 128, 144, 1)
(2892, 4, 128, 144, 1)


In [3]:
x_train2 = np.concatenate((np.zeros((13496,1,128,144,1)), y_train[:, 0:3, :, :,:]), axis=1)
x_val2 = np.concatenate((np.zeros((2892,1,128,144,1)), y_val[:, 0:3, :, :,:]), axis=1)

In [6]:
"""
3-layer stacked ConvLSTM2D Encoder-Decoder
"""

def lstm(n_filter, filter_size):
    # define training encoder
    encoder_inputs = Input(shape=(None, 128, 144, 1))
    encoder_1 = ConvLSTM2D(filters = n_filter, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    encoder_2 = ConvLSTM2D(filters = 32, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    encoder_3 = ConvLSTM2D(filters = 32, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    encoder_outputs_1, encoder_state_h_1, encoder_state_c_1 = encoder_1(encoder_inputs)
    encoder_outputs_2, encoder_state_h_2, encoder_state_c_2 = encoder_2(encoder_outputs_1)
    encoder_outputs_3, encoder_state_h_3, encoder_state_c_3 = encoder_3(encoder_outputs_2)
    # define training decoder
    decoder_inputs = Input(shape=(None, 128, 144, 1))
    decoder_1 = ConvLSTM2D(filters=n_filter, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    decoder_2 = ConvLSTM2D(filters=32, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    decoder_3 = ConvLSTM2D(filters=32, kernel_size=filter_size, padding='same', return_sequences=True, return_state=True,
                           kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    decoder_outputs_1, _, _ = decoder_1([decoder_inputs, encoder_state_h_1, encoder_state_c_1])
    decoder_outputs_2, _, _ = decoder_2([decoder_outputs_1, encoder_state_h_2, encoder_state_c_2])
    decoder_outputs_3, _, _ = decoder_3([decoder_outputs_2, encoder_state_h_3, encoder_state_c_3])
    decoder_conv3d = Conv3D(filters=1, kernel_size=(1,1,32), padding='same', data_format='channels_last',
                            kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))
    decoder_outputs = decoder_conv3d(decoder_outputs_3)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    #print(model.summary(line_length=250))
    
    # define inference encoder
    encoder_model = Model(encoder_inputs, 
                          [encoder_state_h_1, encoder_state_c_1, encoder_state_h_2, encoder_state_c_2, encoder_state_h_3, encoder_state_c_3])
    
    # define inference decoder
    decoder_state_input_h_1 = Input(shape=(128, 144,n_filter))
    decoder_state_input_c_1 = Input(shape=(128, 144,n_filter))
    decoder_state_input_h_2 = Input(shape=(128, 144,32))
    decoder_state_input_c_2 = Input(shape=(128, 144,32))
    decoder_state_input_h_3 = Input(shape=(128, 144,32))
    decoder_state_input_c_3 = Input(shape=(128, 144,32))
    decoder_output_1, decoder_state_h_1_new, decoder_state_c_1_new = decoder_1([decoder_inputs, decoder_state_input_h_1, decoder_state_input_c_1])
    decoder_output_2, decoder_state_h_2_new, decoder_state_c_2_new = decoder_2([decoder_output_1, decoder_state_input_h_2, decoder_state_input_c_2])
    decoder_output_3, decoder_state_h_3_new, decoder_state_c_3_new = decoder_3([decoder_output_2, decoder_state_input_h_3, decoder_state_input_c_3])
    decoder_output = decoder_conv3d(decoder_output_3)
    decoder_model = Model([decoder_inputs , decoder_state_input_h_1 , decoder_state_input_c_1, decoder_state_input_h_2 , decoder_state_input_c_2, 
                           decoder_state_input_h_3 , decoder_state_input_c_3],
                          [decoder_output, decoder_state_h_1_new, decoder_state_c_1_new, decoder_state_h_2_new, decoder_state_c_2_new, 
                           decoder_state_h_3_new, decoder_state_c_3_new])
    
    return model, encoder_model, decoder_model

In [7]:
model, encoder, decoder = lstm(n_filter=64, filter_size=5) #probar filter_size=3

In [8]:
print(model.summary(line_length=135))

Model: "model"
_______________________________________________________________________________________________________________________________________
Layer (type)                                Output Shape                  Param #         Connected to                                 
input_1 (InputLayer)                        [(None, None, 128, 144, 1)]   0                                                            
_______________________________________________________________________________________________________________________________________
input_2 (InputLayer)                        [(None, None, 128, 144, 1)]   0                                                            
_______________________________________________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)                   [(None, None, 128, 144, 64),  416256          input_1[0][0]                                
_________________________________

In [9]:
model.compile(loss='mse', optimizer='adam', metrics=['mae']) #probar sgd

In [ ]:
history = model.fit([x_train, x_train2], y_train, batch_size=4, validation_data=([x_val, x_val2],y_val), epochs=10)

Epoch 1/10
 231/3374 [=>............................] - ETA: 3:12:16 - loss: 2.6759 - mae: 0.4434

In [ ]:
with open('lstm-03', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
history = pickle.load( open( "lstm-03", "rb" ) )

print("~ lstm ~")
print(f"val_loss: {history['val_loss'][-1]}")
print(f"loss: {history['loss'][-1]}")
#print(history)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.ylabel('mean absolute error')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
model.save('model-03.h5')
encoder.save('encoder-03.h5')
decoder.save('decoder-03.h5')